In [3]:
from cryptography.hazmat.primitives import hashes, hmac
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
import cryptography.exceptions
from cryptography.hazmat.primitives.asymmetric import dsa
from cryptography.hazmat.primitives.ciphers.aead import AESGCM

In [4]:
class Emitter:
    def __init__(self, parameters):
        self.private_key = None
        self.public_key = None
        self.shared_key = None
        self.parameters = parameters
        self.get_private_key(parameters)
        self.get_public_key()


    def get_private_key(self, parameters):
        self.private_key = parameters.generate_private_key()

    def get_public_key(self):
        self.public_key = self.private_key.public_key()


    #Criar a chave partilhada, gerada através da chaves pública do receiver e privada do emitter
    def get_derived_key(self, receiver_public_key):
        key_interm = self.private_key.exchange(receiver_public_key)
        # Derivar chave
        self.shared_key = HKDF(
            algorithm=hashes.SHA256(),
            length=32,
            salt=None,
            info=None,
        ).derive(key_interm)

    
    def authentication(self, message):
        #Criar mac_key para cifrar a assinatura definida 
        h = hmac.HMAC(self.shared_key, hashes.SHA256())
        h.update(message)
        return h.finalize()


    def encode(self, data, ad):
        #codificar o texto 
        nonce = os.urandom(16)
        aesgcm = AESGCM(self.shared_key)
        ad += nonce
        #Cifrar com o nounce e metadados usando AESGCM
        ciphertext = aesgcm.encrypt(nonce, data, ad)
        return nonce, ciphertext
    
    def sendMessage(self, mensagem, sigMess):
        #calculos assinatura
        sign = self.authentication(sigMess)
        #gerar um valor aleatorio para os metadados
        ad = os.urandom(12)
        #cifrar o texto e obter o nounce com que foi cifrado.
        nonce, ciphertext = self.encode(mensagem, ad)
        print("Texto cifrado:")
        print(ciphertext)
        #assinatura + nonce + metadados + texto cifrado
        return sign + nonce + ad + ciphertext


